In [1]:
import numpy as np
import cv2 as cv
from ultralytics import YOLO


In [2]:
from IPython import display
from IPython.display import display, Image
import glob

In [3]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)


2.7.1
0.22.1


In [4]:
torch.cuda.is_available()

False

In [5]:
model = YOLO("yolov8m.pt")
model.info()

YOLOv8m summary: 169 layers, 25,902,640 parameters, 0 gradients, 79.3 GFLOPs


(169, 25902640, 0, 79.3204224)

In [6]:
results = model.train(data="annotated/data_v2/data.yaml", epochs=60, imgsz=640, batch=8)

Ultralytics 8.3.153 🚀 Python-3.12.6 torch-2.7.1 CPU (Apple M3 Max)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=annotated/data_v2/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=Fal

RuntimeError: Dataset 'annotated/data_v2/data.yaml' error ❌ 'annotated/data_v2/data.yaml' does not exist

In [ ]:
Image(filename=f'runs/detect/train/confusion_matrix.png', width=800)

In [ ]:
Image(filename=f'runs/detect/train/results.png', width=800)

In [ ]:
!yolo task=detect mode=val model=runs/detect/train/weights/best.pt data=annotated/data_v2/data.yaml

# run predictions

In [7]:
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt source=unlabeled_data/ conf=0.6

Ultralytics 8.3.153 🚀 Python-3.12.6 torch-2.7.1 CPU (Apple M3 Max)
Model summary (fused): 92 layers, 25,864,078 parameters, 0 gradients, 78.8 GFLOPs

Traceback (most recent call last):
  File "/Users/justin.koida/Documents/GitHub/school/Mahjong_Shifu_Project/venv/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/Users/justin.koida/Documents/GitHub/school/Mahjong_Shifu_Project/venv/lib/python3.12/site-packages/ultralytics/cfg/__init__.py", line 983, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/justin.koida/Documents/GitHub/school/Mahjong_Shifu_Project/venv/lib/python3.12/site-packages/ultralytics/engine/model.py", line 555, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/justin.koida/Documents/GitHub/school/Mahjong_Sh

In [ ]:
for image_path in glob.glob(f'runs/detect/predict3/*.jpg'):
    print(image_path)
    display(Image(filename=image_path, height=600))
    print("\n")

# active learning

In [4]:
from PIL import Image, ImageFilter, ImageEnhance
import matplotlib.pyplot as plt
import matplotlib.image as image
import ipywidgets as widgets
from IPython.display import display, clear_output
import glob
import os
import shutil

In [ ]:
MODEL_DIR = "train2"
#This one is the one for the unlabled data directory
DATA = "active_learning/unlabeled_data/*.jpg"

NEW_TRAIN_DIR = "annotated/data_v4/new_train"
MUST_ANNOTATE_DIR = "annotated/data_v4/review_annotate"
cur_length = len(os.listdir("annotated/data_v4/new_train/images"))

In [97]:
check_model = YOLO(f"runs/detect/{MODEL_DIR}/weights/best.pt")
file_paths = glob.glob(DATA)

In [48]:
file_paths

['active_learning/unlabeled_data/aug_img15.jpg',
 'active_learning/unlabeled_data/aug_img5.jpg',
 'active_learning/unlabeled_data/aug_img3.jpg']

In [ ]:
add_button = widgets.Button(description="Yes Add!")
skip_button = widgets.Button(description="Skip")
must_annotate_button = widgets.Button(description="Mannual")
output = widgets.Output()
selected_for_annotation = []
image_index = 0

In [ ]:
def show_image(index):
    base = os.path.basename(file_paths[index])
    save_path = f"active_learning/unlabeled_labeled/{base}"
    with output:
        clear_output(wait=True)
        result = check_model(file_paths[index], conf=0.7)
        result[0].save(save_path)
        img = image.imread(save_path)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Image {index + 1} of {len(file_paths)}")
        plt.show()


def save_yolo_txt(result, label_path):
    img_h, img_w = result.orig_shape
    with open(label_path, "w") as f:
        for box, class_label, con in zip(result.boxes.xyxy, result.boxes.cls, result.boxes.conf):
            #making into yolo format
            x1, y1, x2, y2 = box.tolist()
            x_c = ((x1 + x2) / 2) / img_w
            y_c = ((y1 + y2) / 2) / img_h
            w_norm = (x2 - x1) / img_w
            h_norm = (y2 - y1) / img_h
            f.write(f"{int(class_label)} {x_c:.6f} {y_c:.6f} {w_norm:.6f} {h_norm:.6f}\n")


def on_add_clicked(b):
    global image_index
    current_image_path = file_paths[image_index]
    #save yolo text
    results = check_model(current_image_path, conf=0.7)[0]

    img_dir = os.path.dirname(current_image_path)
    #label_dir = os.path.join(img_dir, "labels")
    label_dir = NEW_TRAIN_DIR + "/labels"
    os.makedirs(label_dir, exist_ok=True)
    label_path = os.path.join(label_dir, os.path.splitext(os.path.basename(current_image_path))[0] + ".txt")
    #label_path = NEW_TRAIN_DIR
    save_yolo_txt(results, label_path)


    os.makedirs(os.path.join(NEW_TRAIN_DIR, "images"), exist_ok=True)
    dst = os.path.join(NEW_TRAIN_DIR, "images", os.path.basename(current_image_path))
    shutil.move(current_image_path, dst)
    file_paths.pop(image_index)
    #selected_for_annotation.append()
    advance()

def on_skip_clicked(b):
    advance()

def on_must_annotate_clicked(b):
    global image_index
    current_image_path = file_paths[image_index]

    os.makedirs(os.path.join(MUST_ANNOTATE_DIR, "images"), exist_ok=True)
    dst = os.path.join(MUST_ANNOTATE_DIR, "images", os.path.basename(current_image_path))
    shutil.move(current_image_path, dst)
    file_paths.pop(image_index)
    advance()

def advance():
    global image_index
    image_index += 1
    if image_index < len(file_paths):
        show_image(image_index)
    else:
        with output:
            clear_output()
            print("All images reviewed.")

In [101]:
add_button.on_click(on_add_clicked)
skip_button.on_click(on_skip_clicked)
must_annotate_button.on_click(on_must_annotate_clicked)

In [102]:
display(widgets.HBox([add_button, skip_button, must_annotate_button]), output)
show_image(image_index)

Output()

In [ ]:
selected_for_annotation

In [ ]:
"""
unlabeled_to_labeled is the directory used to store the current images we want to label
unlabeled_archive is the directory used to store all previous images that required labeling
unlabeled_to_labeled directory content should be deleted on every run of this active learning
"""
for file in selected_for_annotation:
    shutil.copy(file, 'active_learning/unlabeled_to_label')
    shutil.copy(file, 'active_learning/unlabeled_archive')
    os.remove(file)

# run model on new dataset

In [ ]:
FILE_WEIGHTS = "runs/detect/train/weights/best.pt"

In [ ]:
retrain_model = YOLO(FILE_WEIGHTS)

In [ ]:
results = retrain_model.train(data="annotated/data_v3/data.yaml", epochs=60, imgsz=640, batch=8)

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train2/weights/best.pt source=active_learning/unlabeled_data/ conf=0.8

In [ ]:
for image_path in glob.glob(f'runs/detect/predict4/*.jpg'):
    print(image_path)
    display(Image(filename=image_path, height=600))
    print("\n")

# run model live

In [3]:
trained_mahjong = YOLO('runs/detect/train2/weights/best.pt')

In [4]:
results=trained_mahjong(source=0,show=True, conf=0.6)


1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)

WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 (no detections), 227.5ms
0: 480x640 (no detections), 8.2ms
0: 480x640 (no detections), 7.3ms
0: 480x640 (no detections), 6.8ms
0: 480x640 (no detections), 8.0ms
0: 480x640 (no detections), 7.5ms
0: 480x640 (no detections), 6.9ms
0: 480x640 (no detections), 7.1ms
0: 480x640 (no detections), 7.9ms
0: 480x640 (no detections), 8.8ms
0: 480x640 (no detections), 7.7ms
0: 480x640 (no detec

# old code

from 
https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html

In [ ]:
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Display the resulting frame
    cv.imshow('frame', gray)
    if cv.waitKey(1) == ord('q'):
        break
 
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()